In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from imblearn.over_sampling import SMOTE
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Link notebook with google drive and access data from your personal Gdrive
from google.colab import drive
#Mount Google Drive
drive.mount('/content/gdrive')

#Set the data path for dataset and model location (ex: model_loc = "/content/gdrive/My Drive/Dataset/")
dataset_dir = "/content/gdrive/My Drive/Bio Modelling/data/"
model_loc = "/content/gdrive/My Drive/Bio Modelling/data/"

print(os.listdir(dataset_dir))

Mounted at /content/gdrive
['BRCA_label.csv', 'BRCA_miRNA.csv', 'BRCA_DNA_Methylation.csv', 'BRCA_mRNA.csv', 'BRCA_Copy Nunber Variation.csv']


In [ ]:
data = pd.read_csv(dataset_dir+'BRCA_DNA_Methylation.csv',index_col=0)
data

,TCGA.3C.AAAU.01,TCGA.3C.AALI.01,TCGA.3C.AALJ.01,TCGA.3C.AALK.01,TCGA.5L.AAT0.01,TCGA.5T.A9QA.01,TCGA.A1.A0SB.01,TCGA.A1.A0SE.01,TCGA.A1.A0SF.01,TCGA.A1.A0SG.01,...,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01,TCGA.V7.A7HQ.01,TCGA.W8.A86G.01,TCGA.WT.AB41.01,TCGA.WT.AB44.01,TCGA.XX.A899.01,TCGA.XX.A89A.01,TCGA.Z7.A8R5.01,TCGA.Z7.A8R6.01
A1BG,-1.211396,0.974531,0.633283,0.354642,-1.939963,1.328608,0.946010,0.930281,-0.519615,0.554409,...,-0.760042,1.159665,0.333349,0.775567,-0.778884,0.517100,0.177384,-0.380611,0.811548,-0.819564
A1CF,-1.801239,-1.091976,-0.258825,0.344929,-0.018208,-1.007907,1.246215,0.360814,-0.426866,-1.357345,...,-0.582298,-2.142905,-0.306365,0.816486,-0.499731,0.824666,0.807188,-0.027147,0.644539,-2.004627
A2M,-0.696763,-1.597015,-0.873714,-1.080573,-0.788607,-0.583001,-0.654375,0.302439,-1.561816,-0.836623,...,-1.001059,-2.915892,0.635853,0.027964,-1.145160,-0.326970,0.569131,-2.613169,-0.507579,-0.339844
A2ML1,0.832338,0.758754,0.753608,0.548382,0.589277,0.870132,-1.878898,0.160075,0.066117,-0.338768,...,0.892903,-2.480427,0.319169,-0.098838,0.830086,0.617726,0.551416,0.101744,0.269641,-0.094397
A3GALT2,0.857098,0.450506,0.821330,0.497375,0.400632,1.183863,-3.316770,0.227579,0.051009,0.300039,...,1.161783,-1.863239,0.350236,-0.879849,-0.098983,0.524779,0.041604,0.537643,-0.322718,0.007108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,1.079773,1.107008,1.143368,1.327138,0.825476,1.014806,-0.726589,0.116715,-0.967701,-0.634965,...,0.501544,1.248136,-0.609775,0.259632,1.086309,1.867683,0.048509,-0.122038,0.896755,-0.756927
ZYX,0.780058,0.404568,-0.583163,0.047150,0.272223,1.304881,-0.084193,-0.078364,-0.585443,-0.713563,...,0.578764,1.092554,0.706154,-0.837710,1.021682,1.258202,-0.639635,-0.988974,1.156913,-1.043962
ZZEF1,0.747015,0.107306,0.034379,0.014622,0.298979,1.151613,-0.621557,-0.887355,-0.625655,0.094450,...,0.277770,0.629951,0.062128,-0.269128,0.820279,0.573421,-0.542603,-0.854843,0.898579,-0.878013
ZZZ3,0.378339,0.271888,-0.170242,0.484366,0.029261,0.970602,-0.468274,-0.221699,-0.738370,-0.381204,...,0.781932,0.620894,-0.375671,-0.597399,0.227439,1.354182,-0.548846,-0.365477,-0.098397,-0.763912


In [ ]:
label = pd.read_csv(dataset_dir+'BRCA_label.csv')
label

,Label
0,LumA
1,Her2
2,LumB
3,LumA
4,LumA
...,...
666,LumA
667,LumA
668,LumA
669,LumA


In [ ]:
#replace label with numerical data
label['Label'].replace('Normal', 0 , inplace=True)
label['Label'].replace('LumA', 1 , inplace=True)
label['Label'].replace('LumB', 2 , inplace=True)
label['Label'].replace('Basal', 3 , inplace=True)
label['Label'].replace('Her2', 4 , inplace=True)

In [ ]:
# Transpose data to match with label
data = data. T
data

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk
TCGA.3C.AAAU.01,-1.211396,-1.801239,-0.696763,0.832338,0.857098,-0.695053,-0.258891,0.601832,-0.100803,-5.232067,...,0.835761,-2.172061,-0.184735,0.821827,-0.842599,1.079773,0.780058,0.747015,0.378339,0.032275
TCGA.3C.AALI.01,0.974531,-1.091976,-1.597015,0.758754,0.450506,1.570301,0.330579,-0.912745,-0.682832,-1.232277,...,0.664052,-0.897618,-1.043726,0.746737,-0.188174,1.107008,0.404568,0.107306,0.271888,-0.147645
TCGA.3C.AALJ.01,0.633283,-0.258825,-0.873714,0.753608,0.821330,-0.369450,-0.315881,0.098281,-5.269406,-2.441139,...,-0.092097,-0.403899,0.163509,0.991162,-0.337345,1.143368,-0.583163,0.034379,-0.170242,0.748945
TCGA.3C.AALK.01,0.354642,0.344929,-1.080573,0.548382,0.497375,1.342464,-0.518794,-0.274796,-0.187178,-0.232070,...,0.277971,0.752473,0.722295,0.352692,-0.606283,1.327138,0.047150,0.014622,0.484366,0.173110
TCGA.5L.AAT0.01,-1.939963,-0.018208,-0.788607,0.589277,0.400632,1.553188,-0.299152,0.391160,-0.410440,-1.676137,...,-0.008991,0.693802,0.211610,0.025049,-0.391619,0.825476,0.272223,0.298979,0.029261,-0.424061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44.01,0.517100,0.824666,-0.326970,0.617726,0.524779,0.887775,0.553903,0.059472,0.150010,-0.999820,...,0.419641,-0.977392,-1.059823,0.535427,-0.456112,1.867683,1.258202,0.573421,1.354182,0.268539
TCGA.XX.A899.01,0.177384,0.807188,0.569131,0.551416,0.041604,0.890293,0.012863,-1.045639,0.102895,0.209390,...,-0.279068,0.441245,0.352598,-0.305327,-0.071896,0.048509,-0.639635,-0.542603,-0.548846,-0.346807
TCGA.XX.A89A.01,-0.380611,-0.027147,-2.613169,0.101744,0.537643,-0.910491,-1.371328,-0.607679,0.085117,-1.533629,...,-0.331514,-0.125554,-1.163608,-0.518765,0.355571,-0.122038,-0.988974,-0.854843,-0.365477,-0.360622
TCGA.Z7.A8R5.01,0.811548,0.644539,-0.507579,0.269641,-0.322718,0.886293,0.555758,-0.144756,-0.187391,-0.931151,...,0.528904,0.719474,0.967316,0.049875,-0.178119,0.896755,1.156913,0.898579,-0.098397,1.970558


In [ ]:
# Ensure the number of rows in data match with label
if data.shape[0] == label.shape[0]:
    # Add the new column to the transposed DataFrame
    data['Label'] = label['Label'].values
else:
    raise ValueError("The number of rows in df2 must match the number of columns in the transposed df1.")


In [ ]:
data

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk,Label
TCGA.3C.AAAU.01,-1.211396,-1.801239,-0.696763,0.832338,0.857098,-0.695053,-0.258891,0.601832,-0.100803,-5.232067,...,-2.172061,-0.184735,0.821827,-0.842599,1.079773,0.780058,0.747015,0.378339,0.032275,1
TCGA.3C.AALI.01,0.974531,-1.091976,-1.597015,0.758754,0.450506,1.570301,0.330579,-0.912745,-0.682832,-1.232277,...,-0.897618,-1.043726,0.746737,-0.188174,1.107008,0.404568,0.107306,0.271888,-0.147645,4
TCGA.3C.AALJ.01,0.633283,-0.258825,-0.873714,0.753608,0.821330,-0.369450,-0.315881,0.098281,-5.269406,-2.441139,...,-0.403899,0.163509,0.991162,-0.337345,1.143368,-0.583163,0.034379,-0.170242,0.748945,2
TCGA.3C.AALK.01,0.354642,0.344929,-1.080573,0.548382,0.497375,1.342464,-0.518794,-0.274796,-0.187178,-0.232070,...,0.752473,0.722295,0.352692,-0.606283,1.327138,0.047150,0.014622,0.484366,0.173110,1
TCGA.5L.AAT0.01,-1.939963,-0.018208,-0.788607,0.589277,0.400632,1.553188,-0.299152,0.391160,-0.410440,-1.676137,...,0.693802,0.211610,0.025049,-0.391619,0.825476,0.272223,0.298979,0.029261,-0.424061,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44.01,0.517100,0.824666,-0.326970,0.617726,0.524779,0.887775,0.553903,0.059472,0.150010,-0.999820,...,-0.977392,-1.059823,0.535427,-0.456112,1.867683,1.258202,0.573421,1.354182,0.268539,1
TCGA.XX.A899.01,0.177384,0.807188,0.569131,0.551416,0.041604,0.890293,0.012863,-1.045639,0.102895,0.209390,...,0.441245,0.352598,-0.305327,-0.071896,0.048509,-0.639635,-0.542603,-0.548846,-0.346807,1
TCGA.XX.A89A.01,-0.380611,-0.027147,-2.613169,0.101744,0.537643,-0.910491,-1.371328,-0.607679,0.085117,-1.533629,...,-0.125554,-1.163608,-0.518765,0.355571,-0.122038,-0.988974,-0.854843,-0.365477,-0.360622,1
TCGA.Z7.A8R5.01,0.811548,0.644539,-0.507579,0.269641,-0.322718,0.886293,0.555758,-0.144756,-0.187391,-0.931151,...,0.719474,0.967316,0.049875,-0.178119,0.896755,1.156913,0.898579,-0.098397,1.970558,1


In [ ]:
# print the total number of data with missing values across all columns
print(data.isna().sum().sum())

0


In [ ]:
# print the total number of data with duplicated values across all columns
print(data.duplicated().sum())

0


In [ ]:
print(data["Label"].value_counts()[0])
print(data["Label"].value_counts()[1])
print(data["Label"].value_counts()[2])
print(data["Label"].value_counts()[3])
print(data["Label"].value_counts()[4])

31
353
132
113
42


In [ ]:
# Separate features and target
y = data['Label'].values
x = data.drop(['Label'],axis=1)

In [ ]:
# Normalization
scaler = MinMaxScaler(feature_range=(-1, 1))
X_normalized = scaler.fit_transform(x)

# Apply the scaler to the DataFrame
x = pd.DataFrame(scaler.fit_transform(X_normalized), index = x.index, columns = x.columns)
x

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk
TCGA.3C.AAAU.01,0.040905,-0.420021,0.335390,0.932236,0.589542,-0.694817,0.352448,-0.529989,0.776087,-0.701358,...,-0.541577,-0.640874,-0.091785,-0.461240,-0.932383,-0.687472,-0.285471,-0.513845,-0.640458,-0.622507
TCGA.3C.AALI.01,0.764607,-0.116690,0.027662,0.910876,0.439674,0.553878,0.542636,-0.903589,0.678537,0.318271,...,-0.581140,-0.253947,-0.328564,-0.481510,-0.750279,-0.684002,-0.389509,-0.676827,-0.665817,-0.670436
TCGA.3C.AALJ.01,0.651629,0.239624,0.274904,0.909382,0.576358,-0.515340,0.334061,-0.654200,-0.090193,0.010108,...,-0.755360,-0.104051,0.004207,-0.415529,-0.791788,-0.679370,-0.663182,-0.695407,-0.771143,-0.431592
TCGA.3C.AALK.01,0.559379,0.497831,0.204195,0.849807,0.456950,0.428291,0.268593,-0.746226,0.761610,0.573245,...,-0.670095,0.247028,0.158235,-0.587879,-0.866624,-0.655960,-0.488540,-0.700441,-0.615200,-0.584989
TCGA.5L.AAT0.01,-0.200304,0.342528,0.303996,0.861678,0.421291,0.544444,0.339458,-0.581956,0.724191,0.205122,...,-0.736212,0.229216,0.017466,-0.676323,-0.806891,-0.719867,-0.426178,-0.627994,-0.723616,-0.744071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44.01,0.613164,0.703000,0.461795,0.869937,0.467051,0.177660,0.614690,-0.663773,0.818125,0.377529,...,-0.637453,-0.278167,-0.333001,-0.538551,-0.824837,-0.587099,-0.152991,-0.558073,-0.407990,-0.559568
TCGA.XX.A899.01,0.500693,0.695526,0.768104,0.850687,0.288956,0.179048,0.440127,-0.936369,0.810228,0.685782,...,-0.798439,0.152538,0.056329,-0.765505,-0.717923,-0.818846,-0.678829,-0.842408,-0.861335,-0.723491
TCGA.XX.A89A.01,0.315956,0.338705,-0.319684,0.720152,0.471793,-0.813569,-0.006471,-0.828338,0.807248,0.241451,...,-0.810523,-0.019544,-0.361609,-0.823121,-0.598975,-0.840572,-0.775621,-0.921959,-0.817652,-0.727171
TCGA.Z7.A8R5.01,0.710648,0.625966,0.400058,0.768891,0.154669,0.176843,0.615288,-0.714150,0.761575,0.395035,...,-0.612279,0.237010,0.225774,-0.669622,-0.747481,-0.710787,-0.181055,-0.475231,-0.754028,-0.106164


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [16]:
print(y_train.tolist().count(0))
print(y_train.tolist().count(1))
print(y_train.tolist().count(2))
print(y_train.tolist().count(3))
print(y_train.tolist().count(4))

20
247
98
78
26


In [ ]:
# Define the SVM model
model = SVC(kernel="linear")

# Define RFE with the SVM model, choosing a number of features to select
rfe = RFE(estimator=model, n_features_to_select=19000)

# Fit RFE on the training data
rfe.fit(X_train, y_train)

# Transform the training and test sets to include only the selected features
x_train_rfe = rfe.transform(X_train)
x_test_rfe = rfe.transform(X_test)

In [ ]:
# Convert the transformed arrays back to DataFrames
# Get the selected feature names
selected_features = X_train.columns[rfe.support_]

X_train_rfe = pd.DataFrame(x_train_rfe, columns=selected_features, index=X_train.index)
X_test_rfe = pd.DataFrame(x_test_rfe, columns=selected_features, index=X_test.index)

In [ ]:
oversample = SMOTE()
X_resampled, y_resampled = oversample.fit_resample(X_train_rfe, y_train)
y_resampled = pd.DataFrame(y_resampled, columns=['Label'])

In [ ]:
print(y_resampled["Label"].value_counts()[0])
print(y_resampled["Label"].value_counts()[1])
print(y_resampled["Label"].value_counts()[2])
print(y_resampled["Label"].value_counts()[3])
print(y_resampled["Label"].value_counts()[4])

247
247
247
247
247


In [ ]:
# Define the architecture parameters
input_dim = X_resampled.shape[1]   # Number of features
hidden1_dim = 256
hidden2_dim = 128
encoding_dim = 100  # Size of the encoded representations

# Build the SDAE model
input_layer = Input(shape=(input_dim,))
hidden1 = Dense(hidden1_dim, activation='relu')(input_layer)
encoded1 = Dense(hidden2_dim, activation='relu')(hidden1)
encoded2 = Dense(encoding_dim, activation='relu')(encoded1)
decoded1 = Dense(hidden2_dim, activation='relu')(encoded2)
decoded2 = Dense(hidden1_dim, activation='relu')(decoded1)
output_layer = Dense(input_dim, activation='linear')(decoded2)

sdae = Model(inputs=input_layer, outputs=output_layer)

# Compile the SDAE model
sdae.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the SDAE
sdae.fit(X_resampled, X_resampled, epochs=250, batch_size=32, shuffle=True, validation_data=(X_test_rfe, X_test_rfe), callbacks=[early_stopping])

# Extract the encoder part for feature extraction
encoder = Model(inputs=input_layer, outputs=encoded2)

# Extract features using the encoder
X_train_encoded = encoder.predict(X_resampled)
X_test_encoded = encoder.predict(X_test_rfe)

Epoch 1/250
39/39 [==============================] - 6s 33ms/step - loss: 0.0938 - val_loss: 0.0802
Epoch 2/250
39/39 [==============================] - 1s 16ms/step - loss: 0.0479 - val_loss: 0.0622
Epoch 3/250
39/39 [==============================] - 0s 12ms/step - loss: 0.0401 - val_loss: 0.0564
Epoch 4/250
39/39 [==============================] - 0s 12ms/step - loss: 0.0369 - val_loss: 0.0540
Epoch 5/250
39/39 [==============================] - 0s 12ms/step - loss: 0.0347 - val_loss: 0.0575
Epoch 6/250
39/39 [==============================] - 0s 12ms/step - loss: 0.0330 - val_loss: 0.0514
Epoch 7/250
39/39 [==============================] - 0s 11ms/step - loss: 0.0311 - val_loss: 0.0504
Epoch 8/250
39/39 [==============================] - 0s 12ms/step - loss: 0.0297 - val_loss: 0.0510
Epoch 9/250
39/39 [==============================] - 0s 12ms/step - loss: 0.0287 - val_loss: 0.0506
Epoch 10/250
39/39 [==============================] - 0s 11ms/step - loss: 0.0276 - val_loss: 0.0518

In [ ]:
# Define a classifier (example: simple MLP classifier)
num_classes = len(np.unique(y_resampled))  # Number of classes in the target

classifier = Sequential([
    Dense(64, activation='relu', input_dim=encoding_dim),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Softmax for multiclass classification
])

# Compile and train the classifier
classifier.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model with early stopping
classifier.fit(X_train_encoded, y_resampled, epochs=250, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/250
31/31 [==============================] - 3s 9ms/step - loss: 5.0055 - accuracy: 0.2530 - val_loss: 2.7014 - val_accuracy: 0.0405
Epoch 2/250
31/31 [==============================] - 0s 4ms/step - loss: 2.0069 - accuracy: 0.3320 - val_loss: 2.1182 - val_accuracy: 0.0769
Epoch 3/250
31/31 [==============================] - 0s 4ms/step - loss: 1.5508 - accuracy: 0.3937 - val_loss: 2.0601 - val_accuracy: 0.0972
Epoch 4/250
31/31 [==============================] - 0s 4ms/step - loss: 1.3341 - accuracy: 0.4737 - val_loss: 2.1121 - val_accuracy: 0.1053
Epoch 5/250
31/31 [==============================] - 0s 4ms/step - loss: 1.3096 - accuracy: 0.4585 - val_loss: 2.1930 - val_accuracy: 0.1053
Epoch 6/250
31/31 [==============================] - 0s 4ms/step - loss: 1.2084 - accuracy: 0.5405 - val_loss: 2.2166 - val_accuracy: 0.1053
Epoch 7/250
31/31 [==============================] - 0s 4ms/step - loss: 1.0660 - accuracy: 0.5911 - val_loss: 2.2532 - val_accuracy: 0.1053
Epoch 8/250
3

In [ ]:
# Predict probabilities
y_pred_prob = classifier.predict(X_test_encoded)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

7/7 [==============================] - 0s 2ms/step
Accuracy: 0.7673267326732673
